In [2]:
import scorecard as sc
df_match_list = sc.match_list()
df_match_list.head(10)

,ID,City,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,Date
0,1312200,Ahmedabad,2022,Final,Rajasthan Royals,Gujarat Titans,Narendra Modi Stadium,Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,None,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,2022-05-29
1,1312199,Ahmedabad,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,Narendra Modi Stadium,Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,None,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon,2022-05-27
2,1312198,Kolkata,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,Eden Gardens,Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,None,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough,2022-05-25
3,1312197,Kolkata,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,Eden Gardens,Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,None,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma,2022-05-24
4,1304116,Mumbai,2022,70,Sunrisers Hyderabad,Punjab Kings,Wankhede Stadium,Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,None,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan,2022-05-22
5,1304115,Mumbai,2022,69,Delhi Capitals,Mumbai Indians,Wankhede Stadium,Mumbai Indians,field,N,Mumbai Indians,Wickets,5.0,None,JJ Bumrah,"['PP Shaw', 'DA Warner', 'MR Marsh', 'RR Pant'...","['Ishan Kishan', 'RG Sharma', 'D Brevis', 'Til...",Nitin Menon,Tapan Sharma,2022-05-21
6,1304114,Mumbai,2022,68,Chennai Super Kings,Rajasthan Royals,Brabourne Stadium,Chennai Super Kings,bat,N,Rajasthan Royals,Wickets,5.0,None,R Ashwin,"['RD Gaikwad', 'DP Conway', 'MM Ali', 'N Jagad...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,NA Patwardhan,2022-05-20
7,1304113,Mumbai,2022,67,Gujarat Titans,Royal Challengers Bangalore,Wankhede Stadium,Gujarat Titans,bat,N,Royal Challengers Bangalore,Wickets,8.0,None,V Kohli,"['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...","['V Kohli', 'F du Plessis', 'GJ Maxwell', 'KD ...",KN Ananthapadmanabhan,GR Sadashiv Iyer,2022-05-19
8,1304112,Mumbai,2022,66,Lucknow Super Giants,Kolkata Knight Riders,Dr DY Patil Sports Academy,Lucknow Super Giants,bat,N,Lucknow Super Giants,Runs,2.0,None,Q de Kock,"['Q de Kock', 'KL Rahul', 'E Lewis', 'DJ Hooda...","['VR Iyer', 'A Tomar', 'N Rana', 'SS Iyer', 'S...",R Pandit,YC Barde,2022-05-18
9,1304111,Mumbai,2022,65,Sunrisers Hyderabad,Mumbai Indians,Wankhede Stadium,Mumbai Indians,field,N,Sunrisers Hyderabad,Runs,3.0,None,RA Tripathi,"['Abhishek Sharma', 'PK Garg', 'RA Tripathi', ...","['RG Sharma', 'Ishan Kishan', 'DR Sams', 'Tila...",CB Gaffaney,N Pandit,2022-05-17
